# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import os
import requests
import json
import glob
import zipfile
from datetime import datetime

In [3]:
#filepath 
os.getcwd()

'/Users/divyaneopaney/citibike-analysis'

In [4]:
# Folder that contains all 2022 CitiBike .zip files
folderpath = r"data/raw"  

# Creating a list with all .zip files in the folder using a list comprehension
filepaths = [
    os.path.join(folderpath, name)
    for name in os.listdir(folderpath)
    if name.endswith(".zip")]
filepaths

['data/raw/202209-citibike-tripdata.zip',
 'data/raw/202201-citibike-tripdata.zip',
 'data/raw/202203-citibike-tripdata.zip',
 'data/raw/202205-citibike-tripdata.zip',
 'data/raw/202207-citibike-tripdata.zip',
 'data/raw/202211-citibike-tripdata.zip',
 'data/raw/202208-citibike-tripdata.zip',
 'data/raw/202202-citibike-tripdata.zip',
 'data/raw/202212-citibike-tripdata.zip',
 'data/raw/202204-citibike-tripdata.zip',
 'data/raw/202210-citibike-tripdata.zip',
 'data/raw/202206-citibike-tripdata.zip']

In [5]:
def read_zip_to_df(zip_path):
    """
    Open a zip file, read ALL CSV files inside it,
    and return one concatenated DataFrame.
    """
    with zipfile.ZipFile(zip_path) as z:
        csv_names = [name for name in z.namelist() if name.endswith(".csv")]
        if not csv_names:
            raise ValueError(f"No CSV file found inside {zip_path}")
        
        zip_concat = pd.DataFrame()
        
        for csv_name in csv_names:
            with z.open(csv_name) as f:
                tmp = pd.read_csv(f)
                zip_concat = pd.concat([zip_concat, tmp], ignore_index=True)
        
        return zip_concat

### How the data import and join code works

#First, I define `folderpath` to point to the folder in my project that contains all of the CitiBike 2022 data files. Then I use a list comprehension:

```python
filepaths = [
    os.path.join(folderpath, name)
    for name in os.listdir(folderpath)
    if name.endswith(".zip")]

In [6]:
df = pd.concat(
    (read_zip_to_df(f) for f in filepaths),
    ignore_index=True)

/var/folders/wf/s0crnzgd66b7pdj9_7_b6mpw0000gn/T/ipykernel_2142/625673620.py:15: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv(f)
/var/folders/wf/s0crnzgd66b7pdj9_7_b6mpw0000gn/T/ipykernel_2142/625673620.py:15: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv(f)
/var/folders/wf/s0crnzgd66b7pdj9_7_b6mpw0000gn/T/ipykernel_2142/625673620.py:15: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv(f)
/var/folders/wf/s0crnzgd66b7pdj9_7_b6mpw0000gn/T/ipykernel_2142/625673620.py:15: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv(f)
/var/folders/wf/s0crnzgd66b7pdj9_7_b6mpw0000gn/T/ipykernel_2142/625673620.py:15: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=

In [7]:
# Defining the NOAA token
Token = "SbxphqBXykrxXPJRhtGIqvsJEnkMsJUK" 

# Base URL
base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?"

# Parameters
params = (
    "datasetid=GHCND"
    "&datatypeid=TAVG"
    "&limit=1000"
    "&stationid=GHCND:USW00014732"
    "&startdate=2022-01-01"
    "&enddate=2022-12-31")
# Combining base URL + parameters
full_url = base_url + params
full_url

'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31'

In [8]:
#### Getting the data from NOAA
r = requests.get(full_url, headers={'token': Token})
print("Status code:", r.status_code)

# Loading the API response as JSON
d = json.loads(r.text)

# Preview structure
d.keys()

Status code: 200


dict_keys(['metadata', 'results'])

In [9]:
# Securing all the items in the response that correspond to TAVG
avg_temps = [item for item in d['results'] if item['datatype'] == 'TAVG']

# Geting only the date field from all average temperature readings
dates_temp = [item['date'] for item in avg_temps]

# Getting the temperature values
temps = [item['value'] for item in avg_temps]

len(avg_temps), len(dates_temp), len(temps)

(365, 365, 365)

In [10]:
#Building the temperature dataframe
df_temps = pd.DataFrame()

# Converting date strings to datetime and temperatures to °C
df_temps['date'] = [datetime.strptime(d_str, "%Y-%m-%dT%H:%M:%S") for d_str in dates_temp]
df_temps['avgTemp'] = [float(v) / 10.0 for v in temps]

# Keeping only the date (no time part)
df_temps['date'] = df_temps['date'].dt.date

df_temps.head()

,date,avgTemp
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


In [11]:
# Convert started_at to datetime and extract date
df['started_at'] = pd.to_datetime(df['started_at'])
df['date'] = df['started_at'].dt.date

df[['started_at', 'date']].head()

,started_at,date
0,2022-09-14 20:37:39.155,2022-09-14
1,2022-09-17 19:21:39.407,2022-09-17
2,2022-09-17 16:49:21.830,2022-09-17
3,2022-09-08 12:27:40.019,2022-09-08
4,2022-09-16 19:00:19.266,2022-09-16


In [12]:
%%time

df_merged = df.merge(
    df_temps,          #weather dataframe
    how='left',
    on='date',
    indicator=True
)

df_merged['_merge'].value_counts(dropna=False)

CPU times: user 12.1 s, sys: 21 s, total: 33.1 s
Wall time: 42.8 s


_merge
both          29838166
left_only          640
right_only           0
Name: count, dtype: int64

In [13]:
#Exporting merged datasets
os.makedirs("data/Processed", exist_ok=True)

merged_path = "data/Processed/citibike_2022_with_weather.csv"

df_merged.to_csv(merged_path, index=False)
merged_path

'data/Processed/citibike_2022_with_weather.csv'